## Algoritmos en PYSPARK BOW.hs
##### Alumna:Milagros Yarahuaman Rojas
#### Cod:171071

#### - BAGOFWORDS

In [4]:
def bagofwords(Doc):
    """cambia la letras a minusculas,
    filtra letra mayores a 2,
    quita espacios en blanco
    Args:
        texto (str): A string.
    
    Returns:
        [[str]]: Arreglo que contiene documentos que son [str]
    """
    x=Doc.strip().lower().split()
    return list(filter(lambda x:len(x)>2,x))

In [28]:
A=["messi messi messi ronaldo ronaldo balon",
          "messi ronaldo futbol futbol futbol"]
Texto=sc.parallelize(A,4)
Texto=Texto.map(bagofwords)
Texto.take(2)

[['messi', 'messi', 'messi', 'ronaldo', 'ronaldo', 'balon'],
 ['messi', 'ronaldo', 'futbol', 'futbol', 'futbol']]

#### - TF

In [5]:
import math
def TF(Doc):
    """Funcion que calcula el tf de un documento 

    Args:
        Doc (str): un Str que es un documento que contiene palabras

    Returns:
        (str,double): un RDD contiene (palabra,double) tupla."""
    #Obtenemos el total de palabras de doc
    total=len(Doc)
    #Lista sin repeticiones
    DocSR=list(set(Doc))
    #Obtenemos el tf para cada palabra
    mapeo=list(map(lambda x:(x,Doc.count(x)),DocSR))
    tf=list(map(lambda x:(x[0],1+math.log10(x[1]/total)),mapeo))
    return (tf)

In [74]:
A=["messi messi messi ronaldo ronaldo balon",
          "messi ronaldo futbol futbol futbol"]
Texto=sc.parallelize(A,4)
Texto=Texto.map(bagofwords)
TextoTF=Texto.map(TF)
TextoTF.take(2)

[[('ronaldo', 0.5228787452803375),
  ('messi', 0.6989700043360187),
  ('balon', 0.22184874961635637)],
 [('ronaldo', 0.30102999566398125),
  ('futbol', 0.7781512503836436),
  ('messi', 0.30102999566398125)]]

#### - IDF

In [6]:
import math
def IDF(Doc,corpus):
    """Funcion que calcula el IDF de un corpus de TF 

    Args:
        Doc [[(str,double)]]: texto TF
        Doc [str]:texto corpus basico procesado con bagofwords

    Returns:
        (str,double): corpus con IDF de cada palabra sin repeticiones"""
    DocP=Doc.flatMap(lambda x:x)
    DocP=DocP.map(lambda x:x[0]).distinct()
    #Total documentos en corpus
    total=corpus.count()
    corpus2=corpus.collect()
    #Extraer cantida de veces de cada palabra en los documentos
    IDF=DocP.map(lambda x:list(map(lambda y:(x,1 if x in y else 0),corpus2)))
    IDF2=IDF.flatMap(lambda x:x)
    IDF2=IDF2.groupByKey().map(lambda x:(x[0],sum(x[1])))
    IDF2=IDF2.map(lambda x:(x[0],math.log10(1+total/x[1])))
    return IDF2

In [131]:
A=["messi messi messi ronaldo ronaldo balon",
          "messi ronaldo futbol futbol futbol"]
Texto=sc.parallelize(A,4)
Texto=Texto.map(bagofwords)
TextoTF=Texto.map(TF)
TextoIDF=IDF(TextoTF,Texto)
TextoIDF.collect()

[('ronaldo', 0.3010299956639812),
 ('messi', 0.3010299956639812),
 ('balon', 0.47712125471966244),
 ('futbol', 0.47712125471966244)]

#### - TFIDF

In [16]:
def RecIDF(word,TextoIDF):
        """Funcion que recuperar el valor IDF de una palabra
        Args:
            word str:palabra a buscar
            Doc [(str,double)]:texto IDF
        Returns:
            double: IDF numerico de una palbara"""
        L=TextoIDF.collect()
        EL=list(filter(lambda x:x[0]==word,L))
        return double(EL[0][1])

In [17]:
import math

def TFIDF(TextoTF,TextoIDF):
    """Funcion que calcula el IDF para cada Doc de TF
    Args:
        Doc [[(str,double)]]: texto TF
        Doc [(str,double)]:texto IDF

    Returns:
        [(str,double)]: texto con el TFIDF de cada palabra en DOC"""
    
    TFIDF=TextoTF.map(lambda x:list(map(lambda y:(y[0],y[1]*RecIDF(y[0],TextoIDF)),x)))
    return  TFIDF

In [15]:
A=["messi messi messi ronaldo ronaldo balon",
          "messi ronaldo futbol futbol futbol"]
Texto=sc.parallelize(A,4)
Texto=Texto.map(bagofwords)
TextoTF=Texto.map(TF)
TextoIDF=IDF(TextoTF,Texto)
TextoTFIDF=TFIDF(TextoTF,TextoIDF)
TextoTFIDF.take(2)

Traceback (most recent call last):
  File "/home/ubuntu/spark-3.2.0-bin-hadoop3.2/python/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/home/ubuntu/spark-3.2.0-bin-hadoop3.2/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/ubuntu/spark-3.2.0-bin-hadoop3.2/python/pyspark/cloudpickle/cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
  File "/home/ubuntu/spark-3.2.0-bin-hadoop3.2/python/pyspark/rdd.py", line 262, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more

PicklingError: Could not serialize object: RuntimeError: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.